# Segunda entrega · Análisis Exploratorio de Datos

**Dataset:** `propiedades_clean.csv`

**Objetivo:** documentar el inventario completo de propiedades (casas, departamentos, lotes, terrenos y más), entender sus patrones de precio/superficie y dejar hipótesis para la etapa de modelado.

**Contenido:** limpieza mínima (sin la columna `cocheras` de API), estadísticas descriptivas, composición del inventario, visualizaciones univariadas y bivariadas (con cruces dorm/baños y tipo/localidad), correlaciones e insights accionables.


In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from IPython.display import display, Markdown
from matplotlib.ticker import FuncFormatter

pd.set_option("display.float_format", "{:,.2f}".format)
sns.set_theme(style="whitegrid", palette="deep")
plt.rcParams["axes.unicode_minus"] = False

DATA_CANDIDATES = [
    Path("include/data/processed/propiedades_clean.csv"),
    Path("../integrador/include/data/processed/propiedades_clean.csv"),
]

for candidate in DATA_CANDIDATES:
    if candidate.exists():
        DATA_PATH = candidate
        break
else:
    raise FileNotFoundError("No se encontró el dataset procesado. Verifica la ruta relativa desde el notebook.")

df_raw = pd.read_csv(DATA_PATH, sep=";", dtype="unicode")
print(f"Registros brutos: {df_raw.shape[0]} | Columnas: {df_raw.shape[1]}")
display(df_raw.head(3))


## 1. Limpieza y resumen numérico

Conversión de tipos, creación de métricas derivadas y primeras estadísticas para conocer escalas y faltantes. Se ignora la columna `cocheras` porque la API trae valores inconsistentes o nulos.


In [ ]:
NUMERIC_VARS = [
    "Precio (USD)",
    "Precio (moneda original)",
    "Superficie Total (m²)",
    "Superficie Cubierta (m²)",
    "Dormitorios",
    "Baños",
    "Cochera (binaria)",
    "Cantidad de Fotos",
    "Precio por m² Total (USD)",
    "Precio por m² Cubierto (USD)",
]

df = df_raw.copy()

numeric_raw = [
    "propiedad_id",
    "prp_pre_dol",
    "prp_pre",
    "banos",
    "dormitorios",
    "cochera",
    "sup_total",
    "sup_cubierta",
    "fotos_cantidad",
    "prp_lat",
    "prp_lng",
]

for col in numeric_raw:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col].astype(str).str.replace(",", ".", regex=False), errors="coerce")

for col in ["prp_alta", "prp_mod"]:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors="coerce")

RENAME = {
    "propiedad_id": "ID Propiedad",
    "tip_desc": "Tipo",
    "prp_dom": "Dirección",
    "loc_desc": "Localidad",
    "pro_desc": "Provincia",
    "con_desc": "Operación",
    "grupo_tip_desc": "Grupo Tipo",
    "prp_pre_dol": "Precio (USD)",
    "prp_pre": "Precio (moneda original)",
    "banos": "Baños",
    "dormitorios": "Dormitorios",
    "cochera": "Cochera (binaria)",
    "sup_total": "Superficie Total (m²)",
    "sup_cubierta": "Superficie Cubierta (m²)",
    "prp_alta": "Fecha Alta",
    "prp_mod": "Fecha Modificación",
    "url_ficha_inmoup": "URL Ficha",
    "prp_lat": "Latitud",
    "prp_lng": "Longitud",
    "fotos_cantidad": "Cantidad de Fotos",
}

df = df.rename(columns=RENAME)

if "cocheras" in df.columns:
    df = df.drop(columns=["cocheras"])

if {"Precio (USD)", "Superficie Total (m²)"}.issubset(df.columns):
    df["Precio por m² Total (USD)"] = df["Precio (USD)"] / df["Superficie Total (m²)"]

if {"Precio (USD)", "Superficie Cubierta (m²)"}.issubset(df.columns):
    df["Precio por m² Cubierto (USD)"] = df["Precio (USD)"] / df["Superficie Cubierta (m²)"]

for col in ["Precio por m² Total (USD)", "Precio por m² Cubierto (USD)"]:
    if col in df.columns:
        df[col] = df[col].replace([np.inf, -np.inf], np.nan)

if "Cochera (binaria)" in df.columns:
    df["Cochera (binaria)"] = df["Cochera (binaria)"].round().clip(lower=0).astype("Int64")

NUMERIC_VARS = [col for col in NUMERIC_VARS if col in df.columns]

print(f"Dataset listo: {df.shape[0]} filas x {df.shape[1]} columnas")
missing_pct = df.isna().mean().sort_values(ascending=False) * 100
display(missing_pct.head(10).to_frame("porcentaje_na"))

df_numeric = df[NUMERIC_VARS].apply(pd.to_numeric, errors="coerce")
numeric_summary = df_numeric.describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.9]).T.rename(columns={"count": "conteo", "mean": "media"})
numeric_summary["missing_%"] = df_numeric.isna().mean() * 100
if {"75%", "25%"}.issubset(numeric_summary.columns):
    numeric_summary["IQR"] = numeric_summary["75%"] - numeric_summary["25%"]
else:
    numeric_summary["IQR"] = np.nan
display(numeric_summary.round(2))

category_tables = {}
for col in ["Tipo", "Grupo Tipo", "Operación", "Localidad", "Provincia"]:
    if col in df.columns:
        counts = df[col].value_counts(dropna=False)
        top = counts.head(10).to_frame("Avisos")
        top["% sobre total"] = (top["Avisos"] / len(df) * 100).round(1)
        category_tables[col] = top
        display(Markdown(f"**Top {col.lower()} (10 principales)**"))
        display(top)

overview_lines = [f"- {len(df):,} avisos con {df.shape[1]} columnas tras estandarizar nombres y tipos."]

if "Precio (USD)" in df_numeric:
    med = df_numeric["Precio (USD)"].median()
    mean = df_numeric["Precio (USD)"].mean()
    p90 = df_numeric["Precio (USD)"].quantile(0.9)
    overview_lines.append(f"- Precio listado mediano USD {med:,.0f} (promedio USD {mean:,.0f}); el 10% superior supera USD {p90:,.0f}.")

if "Superficie Total (m²)" in df_numeric:
    sup_med = df_numeric["Superficie Total (m²)"].median()
    sup_p90 = df_numeric["Superficie Total (m²)"].quantile(0.9)
    overview_lines.append(f"- Superficie total mediana {sup_med:,.0f} m²; el 10% superior alcanza al menos {sup_p90:,.0f} m² (predios grandes).")

if "Cantidad de Fotos" in df_numeric:
    fotos_mean = df_numeric["Cantidad de Fotos"].mean()
    fotos_med = df_numeric["Cantidad de Fotos"].median()
    fotos_p99 = df_numeric["Cantidad de Fotos"].quantile(0.99)
    overview_lines.append(f"- Engagement: promedio {fotos_mean:,.1f} fotos (mediana {fotos_med:,.0f}); el percentil 99 llega a {fotos_p99:,.0f}.")

if "Tipo" in df.columns:
    tipo_counts = df["Tipo"].value_counts(normalize=True)
    if not tipo_counts.empty:
        top_tipo = tipo_counts.index[0]
        top_pct = tipo_counts.iloc[0] * 100
        overview_lines.append(f"- {top_tipo} es la categoría más frecuente ({top_pct:.1f}%); el stock se diversificó con lotes y departamentos.")

if "cocheras" in df_raw.columns:
    overview_lines.append("- Se descartó la columna `cocheras` de la API porque no es confiable; se usa solo `Cochera (binaria)`.")

faltantes_relevantes = missing_pct[missing_pct > 0].head(5)
if not faltantes_relevantes.empty:
    overview_lines.append("- Columnas con NA relevantes: " + ", ".join([f"{col} {pct:.1f}%" for col, pct in faltantes_relevantes.items()]) + ".")

display(Markdown("**Resumen rápido**\n\n" + "\n".join(overview_lines)))


In [ ]:
def format_large_number(value: float) -> str:
    abs_value = abs(value)
    if abs_value >= 1_000_000:
        return f"{value / 1_000_000:.1f}M"
    if abs_value >= 1_000:
        return f"{value / 1_000:.0f}k"
    return f"{value:,.0f}"

currency_formatter = FuncFormatter(lambda value, _: format_large_number(value))

def clip_percentiles(series: pd.Series, low: float = 0.01, high: float = 0.99) -> pd.Series:
    numeric = pd.to_numeric(series, errors="coerce").dropna()
    if numeric.empty:
        return numeric
    lower, upper = numeric.quantile([low, high])
    return numeric.clip(lower=lower, upper=upper)

def binned_trend(x: pd.Series, y: pd.Series, bins: int = 20):
    mask = ~(x.isna() | y.isna())
    X = x[mask]
    Y = y[mask]
    if X.empty:
        return np.array([]), np.array([])
    quantiles = np.linspace(0, 1, bins + 1)
    edges = X.quantile(quantiles).to_numpy()
    mids = []
    meds = []
    for start, stop in zip(edges[:-1], edges[1:]):
        if stop == start:
            continue
        subset = (X >= start) & (X <= stop)
        if subset.sum() == 0:
            continue
        mids.append(X[subset].median())
        meds.append(Y[subset].median())
    return np.array(mids), np.array(meds)


## 2. Composición del inventario

Mix de avisos por tipo, grupo tipológico y operación para dimensionar el mercado disponible.


In [ ]:
if "Tipo" in df.columns:
    tipo_counts = df["Tipo"].value_counts().head(12)
    fig, ax = plt.subplots(figsize=(10, 5))
    sns.barplot(x=tipo_counts.values, y=tipo_counts.index, orient="h", ax=ax, color="#4E79A7")
    ax.set_xlabel("Avisos")
    ax.set_ylabel("Tipo")
    ax.set_title("Distribución de avisos por tipo (top 12)")
    for idx, value in enumerate(tipo_counts.values):
        ax.text(value + max(tipo_counts.values) * 0.01, idx, f"{value / len(df):.0%}", va="center")
    plt.tight_layout()

if "Grupo Tipo" in df.columns:
    grupo_counts = df["Grupo Tipo"].value_counts()
    fig, ax = plt.subplots(figsize=(8, 4))
    sns.barplot(x=grupo_counts.index, y=grupo_counts.values, ax=ax, color="#59A14F")
    ax.set_ylabel("Avisos")
    ax.set_xlabel("")
    ax.set_title("Mix por grupo inmobiliario")
    ax.tick_params(axis="x", labelrotation=30)
    for label in ax.get_xticklabels():
        label.set_horizontalalignment("right")
    plt.tight_layout()

if "Operación" in df.columns:
    op_counts = df["Operación"].value_counts()
    fig, ax = plt.subplots(figsize=(6, 3.5))
    sns.barplot(x=op_counts.index, y=op_counts.values, ax=ax, color="#F28E2B")
    ax.set_ylabel("Avisos")
    ax.set_xlabel("")
    ax.set_title("Operación (venta/alquiler)")
    for idx, value in enumerate(op_counts.values):
        ax.text(idx, value + op_counts.max() * 0.03, f"{value / len(df):.0%}", ha="center")
    plt.tight_layout()





## 3. Variables discretas y engagement

Distribución de dormitorios, baños, cochera binaria y comportamiento de las fotos publicadas.


In [ ]:
notes = []
variables_discretas = [
    ("Dormitorios", [str(i) for i in range(0, 7)] + ["7+"]),
    ("Baños", [str(i) for i in range(0, 7)] + ["7+"]),
    ("Cochera (binaria)", ["Sin cochera", "Con cochera"]),
]

fig, axes = plt.subplots(1, len(variables_discretas), figsize=(16, 4), sharey=True)

for ax, (column, order) in zip(axes, variables_discretas):
    if column not in df.columns:
        ax.axis("off")
        continue

    series = pd.to_numeric(df[column], errors="coerce")

    if column == "Cochera (binaria)":
        categories = series.round().clip(lower=0, upper=1).map({0: "Sin cochera", 1: "Con cochera"}).dropna()
    else:
        categories = series.dropna().clip(upper=7).apply(lambda value: "7+" if value >= 7 else str(int(round(value))))

    counts = categories.value_counts().reindex(order, fill_value=0)
    total = counts.sum() or 1

    sns.barplot(x=counts.index, y=counts.values, ax=ax, color="#1f77b4")
    ax.set_title(column)
    ax.set_xlabel("")
    if ax is axes[0]:
        ax.set_ylabel("Avisos")
    else:
        ax.set_ylabel("")

    ymax = counts.max() or 1
    for idx, value in enumerate(counts.values):
        pct = value / total
        ax.text(idx, value + ymax * 0.03, f"{pct:.0%}", ha="center", va="bottom", fontsize=9)

    dominant_label = counts.idxmax()
    dominant_pct = counts.max() / total
    notes.append(f"- {column}: {dominant_label} concentra {dominant_pct:.0%} de los avisos.")

plt.tight_layout()

if "Cantidad de Fotos" in df.columns:
    fotos = pd.to_numeric(df["Cantidad de Fotos"], errors="coerce").dropna()
    if not fotos.empty:
        p99 = fotos.quantile(0.99)
        med = fotos.median()
        outliers = int((fotos > p99).sum())

        fig, axes = plt.subplots(1, 2, figsize=(14, 4), sharey=True)
        sns.histplot(fotos, bins=50, ax=axes[0], color="#1f77b4")
        axes[0].set_title("Distribución completa")
        axes[0].set_xlabel("Cantidad de fotos")
        axes[0].set_ylabel("Avisos")

        sns.histplot(fotos[fotos <= p99], bins=30, ax=axes[1], color="#ff7f0e")
        axes[1].axvline(med, linestyle="--", color="black", label=f"Mediana = {med:.0f}")
        axes[1].legend()
        axes[1].set_title(f"Recorte <= p99 ({p99:.0f})")
        axes[1].set_xlabel("Cantidad de fotos")
        axes[1].set_ylabel("")
        axes[1].text(0.98, 0.95, f"{outliers} avisos fuera de rango", ha="right", va="top", transform=axes[1].transAxes, fontsize=9)

        plt.tight_layout()

        notes.append(f"- Fotos: mediana {med:.0f} y solo el 1% supera {p99:.0f} imágenes (cola larga controlada).")

if notes:
    display(Markdown("**Lecturas rápidas**\n\n" + "\n".join(notes)))


## 4. Precio vs atributos categóricos

Comparación de precios según tipo, grupo tipológico y localización relevante.


In [ ]:
notes = []

if {"Tipo", "Precio (USD)"}.issubset(df.columns):
    tipo_df = df[["Tipo", "Precio (USD)"]].copy()
    tipo_df["Precio (USD)"] = pd.to_numeric(tipo_df["Precio (USD)"], errors="coerce")
    tipo_df = tipo_df.dropna()
    tipo_medians = tipo_df.groupby("Tipo")["Precio (USD)"].median().sort_values(ascending=False).head(10)
    tipo_df = tipo_df[tipo_df["Tipo"].isin(tipo_medians.index)]
    counts = tipo_df["Tipo"].value_counts()
    label_map = {tipo: f"{tipo} (n={counts[tipo]})" for tipo in tipo_medians.index}
    tipo_df["Tipo (label)"] = tipo_df["Tipo"].map(label_map)
    order = [label_map[tipo] for tipo in tipo_medians.index]

    fig, ax = plt.subplots(figsize=(9, 5.5))
    sns.boxenplot(data=tipo_df, x="Precio (USD)", y="Tipo (label)", order=order, showfliers=False, ax=ax)
    ax.set_xlabel("Precio (USD)")
    ax.set_ylabel("")
    ax.xaxis.set_major_formatter(currency_formatter)
    ax.set_title("Precio (USD) por tipo de propiedad (top 10 mediana)")
    plt.tight_layout()

    notes.append(f"- Tipos top: {tipo_medians.index[0]} ~USD {tipo_medians.iloc[0]:,.0f}; {tipo_medians.index[-1]} ronda USD {tipo_medians.iloc[-1]:,.0f}.")

if {"Grupo Tipo", "Precio (USD)"}.issubset(df.columns):
    grupo_df = df[["Grupo Tipo", "Precio (USD)"]].copy()
    grupo_df["Precio (USD)"] = pd.to_numeric(grupo_df["Precio (USD)"], errors="coerce")
    grupo_df = grupo_df.dropna()
    grupo_medians = grupo_df.groupby("Grupo Tipo")["Precio (USD)"].median().sort_values(ascending=False)

    fig, ax = plt.subplots(figsize=(9, 4.5))
    sns.boxenplot(data=grupo_df, x="Precio (USD)", y="Grupo Tipo", order=grupo_medians.index, showfliers=False, ax=ax, palette="Set3")
    ax.set_xlabel("Precio (USD)")
    ax.set_ylabel("")
    ax.xaxis.set_major_formatter(currency_formatter)
    ax.set_title("Precio (USD) por grupo tipológico")
    plt.tight_layout()

    notes.append(f"- Gap grupos: diferencia ≈ USD {grupo_medians.iloc[0] - grupo_medians.iloc[-1]:,.0f} entre medianas extrema.")

if {"Localidad", "Precio (USD)"}.issubset(df.columns):
    loc_df = df[["Localidad", "Precio (USD)"]].copy()
    loc_df["Precio (USD)"] = pd.to_numeric(loc_df["Precio (USD)"], errors="coerce")
    loc_df = loc_df.dropna()
    top_localidades = df["Localidad"].value_counts().head(10).index
    loc_df = loc_df[loc_df["Localidad"].isin(top_localidades)]
    loc_medians = loc_df.groupby("Localidad")["Precio (USD)"].median().sort_values()
    counts = loc_df["Localidad"].value_counts()
    label_map = {loc: f"{loc} (n={counts[loc]})" for loc in loc_medians.index}
    loc_df["Localidad (label)"] = loc_df["Localidad"].map(label_map)
    order = [label_map[loc] for loc in loc_medians.index]

    fig, ax = plt.subplots(figsize=(9, 5.5))
    sns.boxenplot(data=loc_df, x="Precio (USD)", y="Localidad (label)", order=order, showfliers=False, ax=ax, color="#ff7f0e")
    ax.set_xlabel("Precio (USD)")
    ax.set_ylabel("")
    ax.xaxis.set_major_formatter(currency_formatter)
    ax.set_title("Precio (USD) por localidad (top 10 por frecuencia)")
    plt.tight_layout()

    gap = loc_medians.iloc[-1] - loc_medians.iloc[0]
    notes.append(f"- Localidades: spread de ~USD {gap:,.0f} entre mediana más alta y más baja.")

if notes:
    display(Markdown("**Puntos destacados**\n\n" + "\n".join(notes)))



## 5. Precio vs superficies

Relaciones log-log y segmentadas para detectar elasticidades y diferencias entre grupos.


In [ ]:
notes = []

if {"Superficie Total (m²)", "Precio (USD)", "Tipo"}.issubset(df.columns):
    cols = ["Superficie Total (m²)", "Precio (USD)", "Tipo"]
    total_df = df[cols].dropna(subset=["Superficie Total (m²)", "Precio (USD)"]).copy()
    total_df["Superficie Total (m²)"] = pd.to_numeric(total_df["Superficie Total (m²)"], errors="coerce")
    total_df["Precio (USD)"] = pd.to_numeric(total_df["Precio (USD)"], errors="coerce")
    total_df = total_df.dropna(subset=["Superficie Total (m²)", "Precio (USD)"])
    total_df = total_df[(total_df["Superficie Total (m²)"] > 0) & (total_df["Precio (USD)"] > 0)]

    if not total_df.empty:
        total_df["Superficie bin"] = pd.qcut(total_df["Superficie Total (m²)"], q=20, duplicates="drop")
        summary = total_df.groupby("Superficie bin").agg(
            med_sup=("Superficie Total (m²)", "median"),
            med_precio=("Precio (USD)", "median"),
            p10=("Precio (USD)", lambda x: x.quantile(0.1)),
            p90=("Precio (USD)", lambda x: x.quantile(0.9)),
            cantidad=("Precio (USD)", "size")
        ).reset_index(drop=True)

        fig, ax1 = plt.subplots(figsize=(8, 5))
        ax1.plot(summary["med_sup"], summary["med_precio"], marker="o", color="#1f77b4", label="Mediana")
        ax1.fill_between(summary["med_sup"], summary["p10"], summary["p90"], color="#1f77b4", alpha=0.15, label="P10-P90")
        ax1.set_xscale("log")
        ax1.set_yscale("log")
        ax1.set_xlabel("Superficie Total (m²)")
        ax1.set_ylabel("Precio (USD)")
        ax1.set_title("Precio vs superficie total - mediana y banda de dispersión")
        ax1.legend(loc="upper left")

        ax2 = ax1.twinx()
        ax2.bar(summary["med_sup"], summary["cantidad"], width=summary["med_sup"] * 0.05, alpha=0.2, color="#555555", label="Cantidad")
        ax2.set_ylabel("Cantidad de avisos")
        ax2.set_yscale("log")
        ax2.legend(loc="lower right")
        plt.tight_layout()

        top_types = total_df["Tipo"].value_counts().head(4).index
        fig, ax = plt.subplots(figsize=(8, 5))
        for tipo in top_types:
            subset = total_df[total_df["Tipo"] == tipo]
            if subset.empty:
                continue
            subset = subset.copy()
            subset["Superficie bin"] = pd.qcut(subset["Superficie Total (m²)"], q=10, duplicates="drop")
            sub_summary = subset.groupby("Superficie bin").agg(
                med_sup=("Superficie Total (m²)", "median"),
                med_precio=("Precio (USD)", "median")
            )
            ax.plot(sub_summary["med_sup"], sub_summary["med_precio"], marker="o", label=tipo)
        ax.set_xscale("log")
        ax.set_yscale("log")
        ax.set_xlabel("Superficie Total (m²)")
        ax.set_ylabel("Precio (USD)")
        ax.set_title("Precio mediano por superficie total y tipo (top 4)")
        ax.legend(title="Tipo")
        plt.tight_layout()

        slope = np.polyfit(np.log10(summary["med_sup"]), np.log10(summary["med_precio"]), 1)[0]
        notes.append(f"- Superficie total: elasticidad de la mediana ≈ {slope:.2f}; la curva evidencia economías de escala moderadas.")

if {"Superficie Cubierta (m²)", "Precio (USD)", "Tipo"}.issubset(df.columns):
    cols = ["Superficie Cubierta (m²)", "Precio (USD)", "Tipo"]
    cub_df = df[cols].dropna(subset=["Superficie Cubierta (m²)", "Precio (USD)"] ).copy()
    cub_df["Superficie Cubierta (m²)"] = pd.to_numeric(cub_df["Superficie Cubierta (m²)"], errors="coerce")
    cub_df["Precio (USD)"] = pd.to_numeric(cub_df["Precio (USD)"] , errors="coerce")
    cub_df = cub_df.dropna(subset=["Superficie Cubierta (m²)", "Precio (USD)"])
    cub_df = cub_df[(cub_df["Superficie Cubierta (m²)"] > 0) & (cub_df["Precio (USD)"] > 0)]

    if not cub_df.empty:
        cub_df["Superficie cubierta bin"] = pd.qcut(cub_df["Superficie Cubierta (m²)"], q=15, duplicates="drop")
        summary = cub_df.groupby("Superficie cubierta bin").agg(
            med_sup=("Superficie Cubierta (m²)", "median"),
            med_precio=("Precio (USD)", "median")
        )

        fig, ax = plt.subplots(figsize=(8, 5))
        sns.lineplot(data=summary, x="med_sup", y="med_precio", marker="o", ax=ax, color="#ff7f0e")
        ax.set_xscale("log")
        ax.set_yscale("log")
        ax.set_xlabel("Superficie Cubierta (m²)")
        ax.set_ylabel("Precio (USD)")
        ax.set_title("Precio mediano vs superficie cubierta")
        plt.tight_layout()

        top_types = cub_df["Tipo"].value_counts().head(4).index
        fig, ax = plt.subplots(figsize=(8, 5))
        for tipo in top_types:
            subset = cub_df[cub_df["Tipo"] == tipo].copy()
            if subset.empty:
                continue
            subset["Superficie cubierta bin"] = pd.qcut(subset["Superficie Cubierta (m²)"], q=10, duplicates="drop")
            sub_summary = subset.groupby("Superficie cubierta bin").agg(
                med_sup=("Superficie Cubierta (m²)", "median"),
                med_precio=("Precio (USD)", "median")
            )
            ax.plot(sub_summary["med_sup"], sub_summary["med_precio"], marker="o", label=tipo)
        ax.set_xscale("log")
        ax.set_yscale("log")
        ax.set_xlabel("Superficie Cubierta (m²)")
        ax.set_ylabel("Precio (USD)")
        ax.set_title("Precio mediano vs superficie cubierta por tipo (top 4)")
        ax.legend(title="Tipo")
        plt.tight_layout()

        notes.append("- Superficie cubierta: diferencias entre tipos se ven como desplazamientos verticales más que cambios de pendiente.")

if notes:
    display(Markdown("**Interpretación superficies-precio**<br><br>" + "<br>".join(notes)))


## 6. Relaciones cruzadas entre atributos

Mapas de calor y gráficos categóricos que combinan dormitorios, baños, tipo, localidad y precio para entender mejor las interacciones clave.


In [ ]:
notes = []

if {"Dormitorios", "Baños"}.issubset(df.columns):
    both = df[["Dormitorios", "Baños", "Precio (USD)"]].copy()
    both[["Dormitorios", "Baños", "Precio (USD)"]] = both[["Dormitorios", "Baños", "Precio (USD)"]].apply(pd.to_numeric, errors="coerce")
    both = both.dropna(subset=["Dormitorios", "Baños"])

    if not both.empty:
        both["Dormitorios clip"] = both["Dormitorios"].clip(upper=6).astype(int)
        both["Baños clip"] = both["Baños"].clip(upper=6).astype(int)

        freq = both.groupby(["Dormitorios clip", "Baños clip"]).size().reset_index(name="Avisos")
        fig, ax = plt.subplots(figsize=(10, 5))
        sns.barplot(data=freq, x="Dormitorios clip", y="Avisos", hue="Baños clip", ax=ax)
        ax.set_xlabel("Dormitorios (clip 6)")
        ax.set_ylabel("Avisos")
        ax.set_title("Avisos por dormitorios y baños")
        plt.tight_layout()

        price_stats = both.groupby(["Dormitorios clip", "Baños clip"]).agg(
            Median=("Precio (USD)", "median"),
            P10=("Precio (USD)", lambda x: x.quantile(0.1)),
            P90=("Precio (USD)", lambda x: x.quantile(0.9))
        ).reset_index()
        fig, ax = plt.subplots(figsize=(10, 5))
        sns.barplot(data=price_stats, x="Dormitorios clip", y="Median", hue="Baños clip", ax=ax)
        ax.yaxis.set_major_formatter(currency_formatter)
        ax.set_xlabel("Dormitorios (clip 6)")
        ax.set_ylabel("Precio mediano (USD)")
        ax.set_title("Precio mediano por dormitorios y baños")
        plt.tight_layout()

        notes.append("- Dormitorios vs baños: barras muestran cómo aumenta la oferta y el precio al sumar ambientes y baños.")

if {"Tipo", "Localidad", "Precio (USD)"}.issubset(df.columns):
    subset = df[["Tipo", "Localidad", "Precio (USD)"]].copy()
    subset["Precio (USD)"] = pd.to_numeric(subset["Precio (USD)"], errors="coerce")
    subset = subset.dropna(subset=["Tipo", "Localidad", "Precio (USD)"])
    if not subset.empty:
        top_tipos = subset["Tipo"].value_counts().head(6).index
        top_locs = subset["Localidad"].value_counts().head(8).index
        subset = subset[subset["Tipo"].isin(top_tipos) & subset["Localidad"].isin(top_locs)]

        count_stats = subset.groupby(["Localidad", "Tipo"]).size().reset_index(name="Avisos")
        fig, ax = plt.subplots(figsize=(12, 5))
        sns.barplot(data=count_stats, x="Localidad", y="Avisos", hue="Tipo", ax=ax)
        ax.set_title("Avisos por localidad y tipo (top)")
        ax.set_xlabel("Localidad")
        ax.set_ylabel("Avisos")
        plt.xticks(rotation=30, ha="right")
        plt.tight_layout()

        price_stats = subset.groupby(["Localidad", "Tipo"]).agg(
            Median=("Precio (USD)", "median"),
            P25=("Precio (USD)", lambda x: x.quantile(0.25)),
            P75=("Precio (USD)", lambda x: x.quantile(0.75))
        ).reset_index()
        fig, ax = plt.subplots(figsize=(12, 5))
        sns.barplot(data=price_stats, x="Localidad", y="Median", hue="Tipo", ax=ax)
        ax.set_title("Precio mediano por localidad y tipo (top)")
        ax.set_xlabel("Localidad")
        ax.set_ylabel("Precio mediano (USD)")
        ax.yaxis.set_major_formatter(currency_formatter)
        plt.xticks(rotation=30, ha="right")
        plt.tight_layout()

        notes.append("- Tipo vs localidad: se ven claramente las localidades premium para cada tipo de propiedad.")

if {"Dormitorios", "Precio (USD)"}.issubset(df.columns):
    price_dorm = df[["Dormitorios", "Precio (USD)"]].apply(pd.to_numeric, errors="coerce").dropna()
    if not price_dorm.empty:
        price_dorm["Dormitorios clip"] = price_dorm["Dormitorios"].clip(upper=6)
        fig, ax = plt.subplots(figsize=(7, 4.5))
        sns.barplot(data=price_dorm, x="Dormitorios clip", y="Precio (USD)", estimator=np.median, color="#4E79A7", ax=ax)
        ax.set_xlabel("Dormitorios (clip 6)")
        ax.set_ylabel("Precio mediano (USD)")
        ax.yaxis.set_major_formatter(currency_formatter)
        ax.set_title("Precio mediano según dormitorios")
        plt.tight_layout()

        notes.append("- Precio vs dormitorios: el escalamiento por ambientes indica segmentos de valor creciente.")

if {"Baños", "Precio (USD)"}.issubset(df.columns):
    price_bath = df[["Baños", "Precio (USD)"]].apply(pd.to_numeric, errors="coerce").dropna()
    if not price_bath.empty:
        price_bath["Baños clip"] = price_bath["Baños"].clip(upper=6)
        fig, ax = plt.subplots(figsize=(7, 4.5))
        sns.barplot(data=price_bath, x="Baños clip", y="Precio (USD)", estimator=np.median, color="#59A14F", ax=ax)
        ax.set_xlabel("Baños (clip 6)")
        ax.set_ylabel("Precio mediano (USD)")
        ax.yaxis.set_major_formatter(currency_formatter)
        ax.set_title("Precio mediano según baños")
        plt.tight_layout()

        notes.append("- Precio vs baños: cada baño adicional desplaza la mediana, marcando propuestas más premium.")

if {"Cochera (binaria)", "Precio (USD)"}.issubset(df.columns):
    price_gar = df[["Cochera (binaria)", "Precio (USD)"]].apply(pd.to_numeric, errors="coerce").dropna()
    if not price_gar.empty:
        price_gar["Cochera label"] = price_gar["Cochera (binaria)"].map({0: "Sin cochera", 1: "Con cochera"})
        fig, ax = plt.subplots(figsize=(6, 4))
        sns.barplot(data=price_gar, x="Cochera label", y="Precio (USD)", estimator=np.median, color="#F28E2B", ax=ax)
        ax.set_xlabel("")
        ax.set_ylabel("Precio mediano (USD)")
        ax.yaxis.set_major_formatter(currency_formatter)
        ax.set_title("Precio mediano según cochera")
        plt.tight_layout()

        medians = price_gar.groupby("Cochera label")["Precio (USD)"].median()
        delta = medians.get("Con cochera", np.nan) - medians.get("Sin cochera", np.nan)
        if pd.notna(delta):
            notes.append(f"- Cochera: prima mediana aproximada de USD {delta:,.0f} por cochera.")

if notes:
    display(Markdown("**Relaciones clave entre atributos**<br><br>" + "<br>".join(notes)))


## 7. Correlaciones multivariadas

Matriz Pearson, ranking Spearman y lectura de pares relevantes.


In [ ]:
notes = []

if NUMERIC_VARS:
    df_numeric = df[NUMERIC_VARS].apply(pd.to_numeric, errors="coerce")
    corr_pearson = df_numeric.corr(method="pearson")
    corr_spearman = df_numeric.corr(method="spearman")

    mask = np.triu(np.ones_like(corr_pearson, dtype=bool))

    fig, ax = plt.subplots(figsize=(10, 8))
    sns.heatmap(
        corr_pearson,
        mask=mask,
        cmap="coolwarm",
        center=0,
        annot=True,
        fmt=".2f",
        linewidths=0.5,
        cbar_kws={"shrink": 0.8},
        ax=ax,
        vmin=-1,
        vmax=1,
        annot_kws={"fontsize": 8},
    )
    ax.set_title("Matriz de correlación (Pearson)")
    plt.tight_layout()

    corr_pairs = (
        corr_spearman.where(~mask)
        .stack()
        .rename("spearman")
        .reset_index()
        .rename(columns={"level_0": "var1", "level_1": "var2"})
    )
    corr_pairs["pearson"] = corr_pairs.apply(lambda row: corr_pearson.loc[row["var1"], row["var2"]], axis=1)
    corr_pairs["abs_spearman"] = corr_pairs["spearman"].abs()
    top_pairs = corr_pairs.sort_values("abs_spearman", ascending=False).head(10)
    display(top_pairs[["var1", "var2", "pearson", "spearman"]])

    if not top_pairs.empty:
        strongest = top_pairs.iloc[0]
        notes.append(f"- Mayor correlación: {strongest['var1']} vs {strongest['var2']} (Spearman {strongest['spearman']:.2f}).")

    key_pairs = {
        "Precio por m² Total (USD)": "Superficie Total (m²)",
        "Precio (USD)": "Dormitorios",
        "Precio (USD)": "Baños",
        "Precio (USD)": "Cochera (binaria)",
    }
    for left, right in key_pairs.items():
        if left in corr_spearman.index and right in corr_spearman.columns:
            spea = corr_spearman.loc[left, right]
            notes.append(f"- {left} vs {right}: Spearman {spea:.2f}.")
else:
    print("No se definieron variables numéricas para el cálculo de correlaciones.")

if notes:
    display(Markdown("**Claves multivariadas**\n\n" + "\n".join(notes)))


## 8. Hipótesis preliminares y próximos pasos

Síntesis de insights, riesgos de calidad y variables prioritarias para modelado.


In [ ]:
hipotesis = []
riesgos = []
variables_clave = [
    "Precio (USD)",
    "Superficie Total (m²)",
    "Superficie Cubierta (m²)",
    "Precio por m² Total (USD)",
    "Precio por m² Cubierto (USD)",
    "Dormitorios",
    "Baños",
    "Cochera (binaria)",
    "Tipo",
    "Grupo Tipo",
    "Localidad",
    "Provincia",
    "Operación",
    "Cantidad de Fotos",
]

if {"Superficie Total (m²)", "Precio (USD)"}.issubset(df.columns):
    data = df[["Superficie Total (m²)", "Precio (USD)"]].apply(pd.to_numeric, errors="coerce").dropna()
    data = data[(data["Superficie Total (m²)"] > 0) & (data["Precio (USD)"] > 0)]
    if len(data) > 50:
        slope = np.polyfit(np.log10(data["Superficie Total (m²)"]), np.log10(data["Precio (USD)"]), 1)[0]
        hipotesis.append(f"- **H1**: el precio crece sublinealmente con la superficie total (elasticidad ≈ {slope:.2f}); conviene modelar precio/m² y segmentar por tipo.")

if {"Dormitorios", "Precio (USD)"}.issubset(df.columns):
    data = df[["Dormitorios", "Precio (USD)"]].apply(pd.to_numeric, errors="coerce").dropna()
    if len(data) > 50:
        corr = data.corr(method="spearman").iloc[0, 1]
        hipotesis.append(f"- **H2**: dormitorios y baños contribuyen al precio pero con rendimientos decrecientes (Spearman dormitorios ≈ {corr:.2f}).")

if {"Cochera (binaria)", "Precio (USD)"}.issubset(df.columns):
    data = df[["Cochera (binaria)", "Precio (USD)"]].apply(pd.to_numeric, errors="coerce").dropna()
    if len(data) > 50:
        corr = data.corr(method="spearman").iloc[0, 1]
        hipotesis.append(f"- **H3**: la presencia de cochera agrega una prima moderada (Spearman ≈ {corr:.2f}); conviene evaluar interacción con tipo y localidad.")

if {"Precio por m² Total (USD)", "Superficie Total (m²)"}.issubset(df.columns):
    data = df[["Precio por m² Total (USD)", "Superficie Total (m²)"]].apply(pd.to_numeric, errors="coerce").dropna()
    if len(data) > 50:
        corr = data.corr(method="spearman").iloc[0, 1]
        hipotesis.append(f"- **H4**: el precio por m² disminuye al crecer la superficie (Spearman ≈ {corr:.2f}); lotes grandes ofrecen descuento por escala.")

if "Grupo Tipo" in df.columns:
    grupo_share = df["Grupo Tipo"].value_counts(normalize=True).head(5)
    if not grupo_share.empty:
        riesgos.append("- Composición: " + ", ".join([f"{idx} {val*100:.1f}%" for idx, val in grupo_share.items()]) + ". Asegurar representatividad al modelar.")

faltantes_clave = {col: missing_pct[col] for col in ["Dormitorios", "Baños", "Superficie Total (m²)", "Cochera (binaria)"] if col in missing_pct.index and missing_pct[col] > 0}
if faltantes_clave:
    riesgos.append("- Faltantes: " + ", ".join([f"{col} {pct:.1f}%" for col, pct in faltantes_clave.items()]) + ". Definir reglas de filtrado/imputación.")

if "Cantidad de Fotos" in df.columns:
    fotos = pd.to_numeric(df["Cantidad de Fotos"], errors="coerce").dropna()
    if not fotos.empty:
        outliers = (fotos > fotos.quantile(0.99)).sum()
        if outliers > 0:
            riesgos.append(f"- Engagement: {outliers} avisos superan el percentil 99 de fotos; considerar winsorización para métricas de marketing.")

variables_disponibles = [var for var in variables_clave if var in df.columns]

if hipotesis:
    display(Markdown("**Hipótesis preliminares**\n\n" + "\n".join(hipotesis)))
if riesgos:
    display(Markdown("**Hallazgos inesperados / calidad de datos**\n\n" + "\n".join(riesgos)))
if variables_disponibles:
    display(Markdown("**Variables candidatas para modelado o segmentación**\n\n" + ", ".join(variables_disponibles)))
